In [14]:
import pandas as pd

dec_merged = pd.read_csv('../DATASET/obs_est_merged/dec_merged.csv')

In [15]:
dec_merged

,lat,lon,year,month,precip_est,precip_obs,bias_dec
0,45.0,-20.0,1982,1,114.242190,158.107760,-43.865570
1,45.0,-19.0,1982,1,109.765625,138.565060,-28.799435
2,45.0,-18.0,1982,1,106.218750,122.867584,-16.648834
3,45.0,-17.0,1982,1,105.335940,109.741210,-4.405270
4,45.0,-16.0,1982,1,103.375000,106.399536,-3.024536
...,...,...,...,...,...,...,...
230251,20.0,16.0,2017,6,0.351885,3.776550,-3.424665
230252,20.0,17.0,2017,6,0.426104,3.719330,-3.293226
230253,20.0,18.0,2017,6,1.090166,3.147125,-2.056959
230254,20.0,19.0,2017,6,1.437823,0.000000,1.437823


In [50]:
dec_average= dec_merged
dec_average = dec_average.drop(columns=['precip_obs'])
dec_average = dec_average[(dec_average['lon'] >= -18) & (dec_average['lon'] <= 0)]
dec_average = dec_average[(dec_average['lat'] >= 20) & (dec_average['lat'] <= 38)]

In [51]:
dec_average[dec_average['lon'] == -18]

,lat,lon,year,month,precip_est,bias_dec
289,38.0,-18.0,1982,1,76.445310,37.123410
330,37.0,-18.0,1982,1,69.164060,34.366390
371,36.0,-18.0,1982,1,65.484375,-3.401425
412,35.0,-18.0,1982,1,67.460940,0.585480
453,34.0,-18.0,1982,1,71.710940,-6.635310
...,...,...,...,...,...,...
230053,24.0,-18.0,2017,6,1.277666,0.705462
230094,23.0,-18.0,2017,6,0.711260,0.253497
230135,22.0,-18.0,2017,6,0.269854,0.040972
230176,21.0,-18.0,2017,6,0.226885,-0.001997


In [52]:
dec_average

,lat,lon,year,month,precip_est,bias_dec
289,38.0,-18.0,1982,1,76.445310,37.123410
290,38.0,-17.0,1982,1,74.929690,21.062350
291,38.0,-16.0,1982,1,72.187500,3.183440
292,38.0,-15.0,1982,1,71.421875,5.347505
293,38.0,-14.0,1982,1,70.515625,11.593811
...,...,...,...,...,...,...
230231,20.0,-4.0,2017,6,0.629229,-2.861219
230232,20.0,-3.0,2017,6,0.605791,-0.138075
230233,20.0,-2.0,2017,6,0.613604,-0.473585
230234,20.0,-1.0,2017,6,0.898760,-3.907758


In [53]:
dec_average.describe()

,lat,lon,year,month,precip_est,bias_dec
count,77976.000000,77976.000000,77976.000000,77976.000000,7.797600e+04,77976.000000
mean,29.000000,-9.000000,1999.500000,3.500000,1.709062e+01,5.001516
std,5.477261,5.477261,10.388361,1.707836,2.151938e+01,18.777963
min,20.000000,-18.000000,1982.000000,1.000000,-3.890422e-19,-350.374760
25%,24.000000,-14.000000,1990.750000,2.000000,1.071596e+00,0.087105
50%,29.000000,-9.000000,1999.500000,3.500000,5.938294e+00,1.611873
75%,34.000000,-4.000000,2008.250000,5.000000,2.896681e+01,10.587353
max,38.000000,0.000000,2017.000000,6.000000,2.026487e+02,162.948240


In [54]:
X, y = dec_average[['lat', 'lon', 'month', 'precip_est']], dec_average['bias_dec'] 
# X = X.sort_values('month')

### Model selection

In [55]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [63]:
# Define the models
models = {
    "Linear Regression": LinearRegression(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "ExtraTrees Regressor": ExtraTreesRegressor(random_state=42),
    "LGBM Regressor": LGBMRegressor(random_state=42)
}
k = 4
kf = KFold(n_splits=k)
results = {}

for model_name, model in models.items():
    train_rmse_scores = []
    test_rmse_scores = []
    train_r2_scores = []
    test_r2_scores = []

    # Note that we pass groups to the split method
    for train_index, test_index in kf.split(X):
        x_train_fold, x_test_fold = X.iloc[train_index].drop(columns=['month']), X.iloc[test_index].drop(columns=['month'])
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        # Train and predict
        model.fit(x_train_fold, y_train_fold)
        y_pred_train = model.predict(x_train_fold)
        y_pred_test = model.predict(x_test_fold)

        # Calculate RMSE
        rmse_train = mean_squared_error(y_train_fold, y_pred_train, squared=False)
        rmse_test = mean_squared_error(y_test_fold, y_pred_test, squared=False)
        r2_train = r2_score(y_train_fold, y_pred_train)
        r2_test = r2_score(y_test_fold, y_pred_test)

        train_rmse_scores.append(rmse_train)
        test_rmse_scores.append(rmse_test)
        train_r2_scores.append(r2_train)
        test_r2_scores.append(r2_test)

    avg_train_rmse = sum(train_rmse_scores) / k
    avg_test_rmse = sum(test_rmse_scores) / k
    avg_train_r2 = sum(train_r2_scores) / k
    avg_test_r2 = sum(test_r2_scores) / k

    results[model_name] = {
        "train_rmse": avg_train_rmse,
        "test_rmse": avg_test_rmse,
        "train_r2": avg_train_r2,
        "test_r2": avg_test_r2
    }

# Optionally, print results
for model_name, metrics in results.items():
    print(f"{model_name} - Train RMSE: {metrics['train_rmse']}, Test RMSE: {metrics['test_rmse']}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 294
[LightGBM] [Info] Number of data points in the train set: 58482, number of used features: 3
[LightGBM] [Info] Start training from score 5.063451
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 294
[LightGBM] [Info] Number of data points in the train set: 58482, number of used features: 3
[LightGBM] [Info] Start training from score 4.500640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000456 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, y

In [64]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

data = results
models = list(data.keys())
train_rmse = [data[model]['train_rmse'] for model in models]
test_rmse = [data[model]['test_rmse'] for model in models]
train_r2 = [data[model]['train_r2'] for model in models]
test_r2 = [data[model]['test_r2'] for model in models]

train_rmse = [round(num, 2) for num in train_rmse]
test_rmse = [round(num, 2) for num in test_rmse]
train_r2 = [round(num, 2) for num in train_r2]
test_r2 = [round(num, 2) for num in test_r2]

standard_deviation = np.std(y)  # Calculate standard deviation using numpy
sample_size = len(y)  # Calculate sample size

standard_error = standard_deviation / np.sqrt(sample_size)

In [65]:
fig = go.Figure()

# Bar chart for R²
fig.add_trace(go.Bar(
    x=models,
    y=train_r2,
    name='Train R²',
    marker_color='blue',
    text=train_r2,  # Add this line to specify the text for each bar
    # 'auto' places the text inside the bars; you can also use 'outside' or 'inside'
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=models,
    y=test_r2,
    name='Test R²',
    marker_color='red',
    text=test_r2,  # Add this line to specify the text for each bar
    # 'auto' places the text inside the bars; you can also use 'outside' or 'inside'
    textposition='auto'
))

# Update the layout
fig.update_layout(
    barmode='group',
    title='R²',
    xaxis_title='Models',
    yaxis_title='Value',
    legend_title='Data',
    width=600,
    # plot_bgcolor='rgba(0,0,0,0)',  # Set plot background color to transparent
    # paper_bgcolor='rgba(0,0,0,0)'
)

# Show the plot
fig.show()

In [58]:
fig = go.Figure()

# Bar chart for RMSE
fig.add_trace(go.Bar(
    x=models,
    y=train_rmse,
    name='Train RMSE',
    marker_color='blue',
    text=train_rmse,  # Add this line to specify the text for each bar
    # 'auto' places the text inside the bars; you can also use 'outside' or 'inside'
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=models,
    y=test_rmse,
    name='Test RMSE',
    marker_color='red',
    text=test_rmse,  # Add this line to specify the text for each bar
    # 'auto' places the text inside the bars; you can also use 'outside' or 'inside'
    textposition='auto'
))
# Update the layout
fig.update_layout(
    barmode='group',
    title='RMSE',
    xaxis_title='Models',
    yaxis_title='Value',
    legend_title='Data',
    width=600,
    # plot_bgcolor='rgba(0,0,0,0)',  # Set plot background color to transparent
    # paper_bgcolor='rgba(0,0,0,0)'
)

# # Line chart for std
# fig.add_trace(go.Scatter(
#     x=models,
#     y=[stdev for model in models],
#     mode='lines+markers',
#     name='Std',
#     line=dict(color='green', width=2)
# ))

# Line chart for std
fig.add_trace(go.Scatter(
    x=models,
    y=[standard_deviation for i in range(len(models))],
    mode='lines+markers',
    name='Std',
    line=dict(color='orange', width=2)
))
fig.show()

### Hyperparams tuning

In [59]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
# from sklearn.metrics import make_scorer, mean_squared_error

# # Define your model
# xgb = LGBMRegressor(random_state=42)

# # Define the parameter grid
# param_grid = {
#     'num_leaves': [31, 41, 51],  # Increase in steps to see the effect
#     'max_depth': [5, 10, 15],  # Adjust based on the complexity of the problem
#     'learning_rate': [0.01, 0.05],  # Small steps to see incremental benefits
#     'n_estimators': [100, 200],  # More trees can be better, but watch for overfitting
#     'subsample': [0.8, 0.9, 1.0],  # Typical subsampling rates
#     'min_child_samples': [20, 30, 40]  # Increasing it can combat overfitting
# }

# # Setup the scoring function
# scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

# #Setup the GroupKFold
# tscv = TimeSeriesSplit(n_splits=3)  # Adjust the number of splits as necessary

# # Setup GridSearchCV with GroupKFold
# grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring=scorer, cv=tscv, verbose=2)

# # Fit the model using groups
# grid_search.fit(X.drop(columns=['month']), y)

# # Get the best estimator and its parameters
# best_xgb = grid_search.best_estimator_
# best_params = grid_search.best_params_

# print("Best parameters:", best_params)
# print("Best RMSE:", -grid_search.best_score_)  # Note: 'best_score_' is negative, so take the negative of it

# # Optionally, use the best model to make predictions or further analysis
# # predictions = best_xgb.predict(X_test)
# # rmse = np.sqrt(mean_squared_error(y_test, predictions))
# # print("Test RMSE:", rmse)

In [90]:
from bayes_opt import BayesianOptimization
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import numpy as np

# Assume 'X' and 'y' are your feature matrix and target variable
kf = KFold(n_splits=4)

# Define your LGBM training function
def LGBM_evaluate(max_depth, n_estimators, learning_rate, colsample_bytree, subsample):
    params = {
        'max_depth': int(max_depth),
        'n_estimators': int(n_estimators),
        'learning_rate': learning_rate,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'random_state': 42
    }
    lgbm = LGBMRegressor(**params)
    # Ensure that X is prepared without needing to drop columns as in your example
    cv_scores = cross_val_score(lgbm, X, y, cv=kf, scoring='neg_root_mean_squared_error')
    return np.mean(cv_scores)

# Set up Bayesian Optimization
optimizer = BayesianOptimization(
    f=LGBM_evaluate,
    pbounds={
        'max_depth': (1, 15),
        'n_estimators': (50, 150),
        'learning_rate': (0.01, 0.2),
        'colsample_bytree': (0.5, 1.0),
        'subsample': (0.5, 1.0)
    },
    random_state=42
)

# Run optimization
optimizer.maximize(init_points=10, n_iter=70)

# Print best parameters
print("Best parameters:", optimizer.max['params'])

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000354 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 301
[LightGBM] [Info] Number of data points in the train set: 58482, number of used features: 4
[LightGBM] [Info] Start training from score 5.063451
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may 

In [91]:
# from bayes_opt import BayesianOptimization
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import TimeSeriesSplit, cross_val_score
# import numpy as np

# # Assume 'y' is your target variable
# tscv = TimeSeriesSplit(n_splits=3) 
# # Define your RandomForest training function
# def RF_evaluate(max_depth, n_estimators, min_samples_split, min_samples_leaf):
#     params = {
#         'max_depth': int(max_depth),
#         'n_estimators': int(n_estimators),
#         'min_samples_split': int(min_samples_split),
#         'min_samples_leaf': int(min_samples_leaf),
#         'random_state': 42
#     }
#     rf = RandomForestRegressor(**params)
#     cv_scores = cross_val_score(rf, X.drop(columns=['month']), y, cv=tscv, scoring='neg_root_mean_squared_error')
#     return np.mean(cv_scores)

# # Set up Bayesian Optimization
# optimizer = BayesianOptimization(
#     f=RF_evaluate,
#     pbounds={
#         'max_depth': (1, 15),
#         'n_estimators': (50, 150),
#         'min_samples_split': (2, 10),
#         'min_samples_leaf': (1, 4)
#     },
#     random_state=42
# )

# # Run optimization
# optimizer.maximize(init_points=10, n_iter=70)

# # Print best parameters
# print("Best parameters:", optimizer.max['params'])

In [61]:
best_params_xgb = {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.8}
best_params_rf = {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
# Create Extra Trees model using the best parameters
# best_model = XGBRegressor(**best_params_xgb, random_state=42)
# best_model = RandomForestRegressor(**best_params_rf, random_state=42)
best_model = LGBMRegressor(random_state=42)

In [77]:
best_model.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 301
[LightGBM] [Info] Number of data points in the train set: 77976, number of used features: 4
[LightGBM] [Info] Start training from score 5.001516


LGBMRegressor(random_state=42)

In [78]:
## testing set
dec_sub = pd.read_csv ('../DATASET/dataset_test2023/dec23.csv')
X_sub = dec_sub[['lat', 'lon', 'month', 'prec_est']]

y_sub = best_model.predict(X_sub)
# dec_sub

In [87]:
submission_dec = pd.DataFrame({ # Example: creating a sequence of IDs from 1 to 781
    'bias': y_sub,
    'lat': dec_sub['lat'],
    'lon': dec_sub['lon'],
    'month': dec_sub['month'],
    'prec_est': dec_sub['prec_est'],
    'corrected_prec': dec_sub['prec_est'] - y_sub
})

In [88]:
submission_dec

,bias,lat,lon,month,prec_est,corrected_prec
0,-16.249827,40.0,-20.0,1,2.864746,19.114573
1,-16.249827,40.0,-19.0,1,2.819092,19.068919
2,-16.249827,40.0,-18.0,1,2.742554,18.992381
3,-16.249827,40.0,-17.0,1,2.552734,18.802561
4,-16.235308,40.0,-16.0,1,2.354126,18.589434
...,...,...,...,...,...,...
5161,-1.154145,20.0,16.0,6,0.010980,1.165125
5162,-1.154145,20.0,17.0,6,0.010370,1.164515
5163,-1.154145,20.0,18.0,6,0.015069,1.169214
5164,-1.154145,20.0,19.0,6,0.014123,1.168268


In [ ]:
# from sklearn.model_selection import TimeSeriesSplit
# from sklearn.metrics import mean_squared_error
# import numpy as np

# final_results = {}

# # Define the number of splits
# tscv = TimeSeriesSplit(n_splits=5)

# # You can iterate over the splits
# for train_index, test_index in tscv.split(X):
#     # Use .iloc for positional indexing
#     x_train_fold, x_test_fold = X.iloc[train_index].drop(columns=['month']), X.iloc[test_index].drop(columns=['month'])
#     y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

#     best_model.fit(x_train_fold, y_train_fold)
#     y_pred_train = best_model.predict(x_train_fold)
#     y_pred_test = best_model.predict(x_test_fold)

#     rmse_train = mean_squared_error(
#         y_train_fold, y_pred_train, squared=False)
#     rmse_test = mean_squared_error(y_test_fold, y_pred_test, squared=False)

#     train_rmse_scores.append(rmse_train)
#     test_rmse_scores.append(rmse_test)

# avg_train_rmse = sum(train_rmse_scores) / k
# avg_test_rmse = sum(test_rmse_scores) / k

# final_results["metrics"] = {
#     "RMSE train": avg_train_rmse,
#     "RMSE test": avg_test_rmse,
# }
# print(final_results)

{'metrics': {'RMSE train': 4.276509256257327, 'RMSE test': 8.32618806517496}}


In [ ]:
# rmse = np.sqrt(mean_squared_error(test['bias_dec'], y_sub))